In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from jax import vmap
import jax.numpy as jnp
from pathlib import Path
import numpy as np
from einops import rearrange
import jax
import matplotlib.pyplot as plt

In [3]:
from colora.data import load_all_hdf5, split_data_by_mu, prepare_coordinate_data
data_dir = Path('./data')
data_path = data_dir / 'burgers'
mus, sols, spacing = load_all_hdf5(data_path)

train_mus = np.asarray([0.001, 0.00199, 0.00298, 0.00496, 0.00595, 0.00694, 0.00892, 0.01])
test_mus = np.asarray([0.00397, 0.00793])
train_sols, test_sols = split_data_by_mu(mus, sols, train_mus, test_mus) # mus X variables X time X space_x X space_y
n_mu_train, n_q, n_t, n_x1, n_x2 = train_sols.shape
n_mu_test, n_q, n_t, n_x1, n_x2 = test_sols.shape

print(f'n_mu train: {n_mu_train}, n_mu test: {n_mu_test}, n_variables: {n_q}, n_time samples: {n_t}, n_x samples: {n_x1}, n_x2 samples: {n_x2}')


n_mu train: 8, n_mu test: 2, n_variables: 1, n_time samples: 51, n_x samples: 129, n_x2 samples: 129


In [4]:
# import glob
# import os

# import h5py

# # downsample data
# n_space, n_time = 2, 2 # downsample in space and time by these factors
# train_sols, test_sols = train_sols[: , :, ::n_time, ::n_space, ::n_space], test_sols[: , :, ::n_time, ::n_space, ::n_space]
# spacing[1] = spacing[1][::n_time]
# spacing[2] = spacing[2][::n_space]
# spacing[3] = spacing[3][::n_space]
# n_mu_train, n_q, n_t, n_x1, n_x2 = train_sols.shape
# n_mu_test, n_q, n_t, n_x1, n_x2 = test_sols.shape
# print(n_mu_train, n_mu_test, n_q, n_t, n_x1, n_x2)

# def make_attach_scale(f, key, dim_i, scale_arr, scale_name=''):
#     # label dimension
#     f['u'].dims[dim_i].label = key
#     # set scale in file
#     f[key] = scale_arr
#     # make scale
#     f[key].make_scale(key)
#     # attach scale to dimension
#     f['u'].dims[dim_i].attach_scale(f[key])


# def save_hdf5(equation, variant, u, scales, info={}, mu=None, path=None):

#     Q = u.shape[0]
#     D = len(u.shape)-2

#     labels = ['q', 't', 'x0', 'x1', 'x2']
#     filename = f'{equation}_{variant}_{D}D_{Q}Q'
#     if path is None:
#         path = './hdf5_data'
#     f = h5py.File(f'{path}/{filename}.hdf5', 'w')
#     f.attrs['equation'] = equation
#     f.attrs['variant'] = variant
#     f.attrs['input_dim'] = D
#     f.attrs['output_dim'] = Q

#     f.create_dataset("u", data=u)

#     for i, (scale, label) in enumerate(zip(scales, labels)):
#         make_attach_scale(f, label, i, scale)

#     if mu is not None:
#         f.attrs['mu'] = mu

#     # add info
#     info_g = f.create_group("info")
#     for k, v in info.items():
#         info_g.attrs[k] = v

#     # clean up
#     f.close()


# for s,mu in zip(test_sols, test_mus):
#     print(s.shape)
#     mu_str = str(mu).replace('.', 'p')
#     variant = f'{mu_str}'
#     save_hdf5('vlasov', variant, s, spacing, mu=mu, path='./')


# for s,mu in zip(train_sols, train_mus):
#     print(s.shape)
#     mu_str = str(mu).replace('.', 'p')
#     variant = f'{mu_str}'
#     save_hdf5('burgers', variant, s, spacing, mu=mu, path='./')

In [5]:
y_train, mu_t_train, X_grid =  prepare_coordinate_data(spacing, train_mus, train_sols)
y_test, mu_t_test, X_grid =  prepare_coordinate_data(spacing, test_mus, test_sols)

In [6]:
def normalize(x, mean, std):
    return (x-mean)/std

mean, std = jnp.mean(mu_t_train, axis=0), jnp.std(mu_t_train, axis=0)
mu_t_train = normalize(mu_t_train, mean, std)
mu_t_test = normalize(mu_t_test, mean, std)


In [7]:
from colora.build import build_colora

key = jax.random.PRNGKey(1)

x_dim = 2
mu_t_dim = 2
u_dim = 1

u_layers = ['P', 'C', 'C', 'C', 'C', 'C', 'C', 'C'] # seven colora layers with 1 alpha each means we will have laten dim of 7
h_layers = ['D', 'D', 'D']
rank = 3

u_hat_config = {'width': 25, 'layers': u_layers}
h_config = {'width': 15, 'layers': h_layers}

u_hat_fn, h_fn, theta_init, psi_init = build_colora(
    u_hat_config, h_config, x_dim, mu_t_dim, u_dim, lora_filter=['alpha'], period=[2.0, 2.0], rank=rank, key=key)

In [8]:

h_v_mu_t = vmap(h_fn, in_axes=(None, 0)) # vmap over mu_t array to generate array of phis
u_hat_v_x =  vmap(u_hat_fn, in_axes=(None, None, 0)) # vmaped over x to generate solution field over space points
u_hat_v_x_phi =  vmap(u_hat_v_x, in_axes=(None, 0, None)) # vmaped over x to generate solution field over space points

In [9]:
def predict(psi_theta, mu_t, X_grid):
    psi, theta = psi_theta
    phis = h_v_mu_t(psi, mu_t)
    pred = u_hat_v_x_phi(theta, phis, X_grid)
    return pred

def relative_loss_fn(psi_theta, mu_t, sols, X_grid):
    pred = predict(psi_theta, mu_t, X_grid)
    loss = jnp.linalg.norm(
        sols - pred, axis=(1,2)) / jnp.linalg.norm(sols, axis=(1,2))
    return loss.mean()



In [10]:
from colora.data import Dataset

# the dataset is just responsible for batching the data over the mu_t tensor 
# in order to aviod memory overflow on the GPU
dataset = Dataset(mu_t_train, X_grid, y_train, n_batches=15, key=key)
dataset = iter(dataset)
def args_fn():
    return next(dataset)

In [11]:
from colora.adam import adam_opt

psi_theta = (psi_init, theta_init)
opt_psi_theta, loss_history = adam_opt(psi_theta, relative_loss_fn, args_fn, steps=100_000, learning_rate=5e-3, verbose=True)
opt_psi, opt_theta = opt_psi_theta


  0%|          | 0/100000 [00:00<?, ?it/s]

In [12]:

pred = predict(opt_psi_theta, mu_t_test, X_grid)
pred = rearrange(pred, '(M T) (N1 N2) Q -> M Q T N1 N2', Q=n_q, T=n_t, N1=n_x1, N2=n_x2)
# y_test = rearrange(y_test, '(M T) (N1 N2) Q -> M Q T N1 N2', Q=n_q, T=n_t, N1=n_x1, N2=n_x2)

In [13]:
test_vec =  rearrange(test_sols, 'M Q T N1 N2 -> M (Q T N1 N2)')
pred_vec =  rearrange(pred, 'M Q T N1 N2 -> M (Q T N1 N2)')
rel_err = np.linalg.norm(test_vec- pred_vec, axis=1)/np.linalg.norm(test_vec, axis=1)
mean_rel_err = rel_err.mean()
print(f'Test mean relative error: {mean_rel_err:.2E}')

Test mean relative error: 1.12E-03


In [30]:
from colora.plot import imshow_movie

imshow_movie(pred[0][0], save_to='./img/burgers.gif', t=spacing[1], title='Burgers', tight=True, live_cbar=True)